# Week 5 Homework 

In this homework we'll put what we learned about Spark in practice.

For this homework we will be using the FHV 2019-10 data found here. [FHV Data](https://github.com/DataTalksClub/nyc-tlc-data/releases/download/fhv/fhv_tripdata_2019-10.csv.gz)

> [!NOTE]  
> 
> Make sure to run [bash script](../../../05-batch/code/download_data.sh) to prepare data before doing homework. We had done for yellow and green for years 2020-2021, when following [Video 5.3.3](https://www.youtube.com/watch?v=CI3P4tAtru4&list=PL3MmuxUbc_hJed7dXYoJw8DoCuVHhGEQb&index=56). But now need the dataset for homework as linked above.

```bash 
./download_data.sh fhv 2019
```

Actually just need the one file for 2019-10 for this homeowork.

In [1]:
import pyspark
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .master("local[*]") \
    .appName('test') \
    .getOrCreate()

your 131072x1 screen size is bogus. expect trouble
24/03/03 17:42:33 WARN Utils: Your hostname, Cinders resolves to a loopback address: 127.0.1.1; using 172.17.156.62 instead (on interface eth0)
24/03/03 17:42:33 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/03/03 17:42:35 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [2]:
spark.version

'3.5.1'

## Question 1: 

### Answer 1: `'3.5.1'`

**Install Spark and PySpark** 

- Install Spark
- Run PySpark
- Create a local spark session
- Execute spark.version.

What's the output?

> [!NOTE]
> 
> To install PySpark follow this [guide](https://github.com/DataTalksClub/data-engineering-zoomcamp/blob/main/05-batch/setup/pyspark.md)

In [3]:
zones_infile = f'../../../data/csv/taxi+_zone_lookup.csv'
zones_outpath = f'../../../data/zones/'
fhv_infile = f'../../../data/raw/fhv/2019/10/fhv_tripdata_2019_10.csv.gz'
fhv_outpath = f'../../../data/pq/fhv/2019/10/'

In [4]:
import pandas as pd
df=pd.read_csv(fhv_infile, nrows=10)

In [5]:
columns = df.columns

In [6]:
df.dtypes

dispatching_base_num       object
pickup_datetime            object
dropOff_datetime           object
PUlocationID                int64
DOlocationID                int64
SR_Flag                   float64
Affiliated_base_number     object
dtype: object

In [7]:
df.head()

,dispatching_base_num,pickup_datetime,dropOff_datetime,PUlocationID,DOlocationID,SR_Flag,Affiliated_base_number
0,B00009,2019-10-01 00:23:00,2019-10-01 00:35:00,264,264,NaN,B00009
1,B00013,2019-10-01 00:11:29,2019-10-01 00:13:22,264,264,NaN,B00013
2,B00014,2019-10-01 00:11:43,2019-10-01 00:37:20,264,264,NaN,B00014
3,B00014,2019-10-01 00:56:29,2019-10-01 00:57:47,264,264,NaN,B00014
4,B00014,2019-10-01 00:23:09,2019-10-01 00:28:27,264,264,NaN,B00014


In [8]:
from pyspark.sql import types

fhv_schema = types.StructType([
    types.StructField("dispatching_base_num", types.StringType(), True),
    types.StructField("pickup_datetime", types.TimestampType(), True),
    types.StructField("dropOff_datetime", types.TimestampType(), True),
    types.StructField("PULocationID", types.IntegerType(), True),
    types.StructField("DOLocationID", types.IntegerType(), True),
    types.StructField("SR_Flag", types.DoubleType(), True),
    types.StructField("Affiliated_base_number", types.StringType(), True),
])

In [9]:
!pwd

/mnt/c/Users/ellabelle/Github/data-engineering-zoomcamp/cohorts/2024/de-lessons


In [10]:
df_fhv_10 = spark.read \
    .option("header", "true") \
    .schema(fhv_schema) \
    .csv(fhv_infile)
    
df_fhv_10 \
    .repartition(6) \
    .write.parquet(fhv_outpath, mode='overwrite')


In [11]:
ls -la ../../../data/pq/fhv/2019/10/*.parquet

-rwxrwxrwx 1 ellabelle ellabelle 6670578 Mar  3 17:42 ../../../data/pq/fhv/2019/10/part-00000-736c78e4-b761-4537-b49c-55f9e45c368d-c000.snappy.parquet*
-rwxrwxrwx 1 ellabelle ellabelle 6660082 Mar  3 17:42 ../../../data/pq/fhv/2019/10/part-00001-736c78e4-b761-4537-b49c-55f9e45c368d-c000.snappy.parquet*
-rwxrwxrwx 1 ellabelle ellabelle 6665599 Mar  3 17:42 ../../../data/pq/fhv/2019/10/part-00002-736c78e4-b761-4537-b49c-55f9e45c368d-c000.snappy.parquet*
-rwxrwxrwx 1 ellabelle ellabelle 6655793 Mar  3 17:42 ../../../data/pq/fhv/2019/10/part-00003-736c78e4-b761-4537-b49c-55f9e45c368d-c000.snappy.parquet*
-rwxrwxrwx 1 ellabelle ellabelle 6664355 Mar  3 17:42 ../../../data/pq/fhv/2019/10/part-00004-736c78e4-b761-4537-b49c-55f9e45c368d-c000.snappy.parquet*
-rwxrwxrwx 1 ellabelle ellabelle 6667744 Mar  3 17:42 ../../../data/pq/fhv/2019/10/part-00005-736c78e4-b761-4537-b49c-55f9e45c368d-c000.snappy.parquet*


## Question 2: 

### Answer 2: `6MB`

**FHV October 2019**

Read the October 2019 FHV into a Spark Dataframe with a schema as we did in the lessons.

Repartition the Dataframe to 6 partitions and save it to parquet.

What is the average size of the Parquet (ending with .parquet extension) Files that were created (in MB)? Select the answer which most closely matches.

- 1MB
- 6MB
- 25MB
- 87MB

In [12]:
from pyspark.sql import functions as F

Read in data 
- from .parquet instead of .csv, 
- into a spark dataframe instead of pandas

In [14]:
df = spark.read.parquet(fhv_outpath)

In [15]:
df.printSchema()

root
 |-- dispatching_base_num: string (nullable = true)
 |-- pickup_datetime: timestamp (nullable = true)
 |-- dropOff_datetime: timestamp (nullable = true)
 |-- PULocationID: integer (nullable = true)
 |-- DOLocationID: integer (nullable = true)
 |-- SR_Flag: double (nullable = true)
 |-- Affiliated_base_number: string (nullable = true)



In [16]:
df\
    .filter(F.to_date(df.pickup_datetime) == '2019-10-15')\
    .count()


62610

## Question 3: 

### Answer 3: `62,610`

**Count records** 

How many taxi trips were there on the 15th of October?

Consider only trips that started on the 15th of October.

- 108,164
- 12,856
- 452,470
- 62,610

> [!IMPORTANT]
> Be aware of columns order when defining schema

Trips showing first 5 dates

In [17]:
df\
    .select('pickup_datetime', 'dropOff_datetime')\
    .orderBy(F.col("pickup_datetime").asc())\
    .show(5)

+-------------------+-------------------+
|    pickup_datetime|   dropOff_datetime|
+-------------------+-------------------+
|2019-10-01 00:00:00|2019-10-01 00:30:00|
|2019-10-01 00:00:00|2019-10-01 00:23:00|
|2019-10-01 00:00:05|2019-10-01 00:24:26|
|2019-10-01 00:00:12|2019-10-02 00:06:06|
|2019-10-01 00:00:12|2019-10-01 00:04:06|
+-------------------+-------------------+
only showing top 5 rows



Trips showing last 5 dates

In [18]:
df\
    .select('pickup_datetime', 'dropOff_datetime')\
    .orderBy(F.col("pickup_datetime").desc())\
    .show(5)

+-------------------+-------------------+
|    pickup_datetime|   dropOff_datetime|
+-------------------+-------------------+
|2019-10-31 23:59:59|2019-11-01 00:08:39|
|2019-10-31 23:59:56|2019-11-01 00:21:34|
|2019-10-31 23:59:54|2019-11-01 00:15:22|
|2019-10-31 23:59:52|2019-11-01 00:11:09|
|2019-10-31 23:59:51|2019-11-01 00:09:32|
+-------------------+-------------------+
only showing top 5 rows



1. [order by, sort pyspark sql syntax](https://sparkbyexamples.com/pyspark/pyspark-orderby-and-sort-explained/)
2. [unix timestamp diff syntax](https://sparkbyexamples.com/pyspark/pyspark-timestamp-difference-seconds-minutes-hours/)

- using built-in `unix_timestamp` function, get the difference of dropoff to pickup
- since answer is in seconds, convert to hours
- select just the 4 columns
- sort by `DiffInHours` column, in descending order
- show top 5 rows

In [19]:
df\
    .withColumn('DiffInSeconds',F.unix_timestamp("dropOff_datetime") - F.unix_timestamp('pickup_datetime')) \
    .withColumn('DiffInHours',F.round(F.col('DiffInSeconds')/3600))\
    .select('pickup_datetime', 'dropOff_datetime','DiffInSeconds','DiffInHours')\
    .orderBy(F.col("DiffInHours").desc())\
    .show(5)

+-------------------+-------------------+-------------+-----------+
|    pickup_datetime|   dropOff_datetime|DiffInSeconds|DiffInHours|
+-------------------+-------------------+-------------+-----------+
|2019-10-28 09:00:00|2091-10-28 09:30:00|   2272149000|   631153.0|
|2019-10-11 18:00:00|2091-10-11 18:30:00|   2272149000|   631153.0|
|2019-10-31 23:46:33|2029-11-01 00:13:00|    315620787|    87672.0|
|2019-10-01 21:43:42|2027-10-01 21:45:23|    252460901|    70128.0|
|2019-10-17 14:00:00|2020-10-18 00:00:00|     31658400|     8794.0|
+-------------------+-------------------+-------------+-----------+
only showing top 5 rows



There cannot be any taxi trips where the drop-off date is in the ***future***!! Obviously there are outliers from errorneous data collection or data entry. Drop-off date of `year` 2091 is so obviously wrong for first entry with `2091-10-11` or `2091-10-28`. Most likely a typo of `2019`, getting transposed.

As there's no instructions to clean or transform data, I'm going to take the answer as-is. The homework question 4 just states `What is the length of the longest trip in the dataset in hours?`

In [20]:
# using built-in `unix_timestamp` function, get the difference of dropoff to pickup
# since above answer is in seconds, convert to hours
# get the max and take the first entry
df.withColumn('DiffInSeconds',F.unix_timestamp("dropOff_datetime") - F.unix_timestamp('pickup_datetime')) \
  .withColumn('DiffInHours', F.col('DiffInSeconds')/3600)\
  .agg(F.max('DiffInHours'))\
  .first()[0]

631152.5

## Question 4: 

### Answer 4: `631,152.50 Hours`

**Longest trip for each day** 

What is the length of the longest trip in the dataset in hours?

- 631,152.50 Hours
- 243.44 Hours
- 7.68 Hours
- 3.32 Hours

## Question 5: 

### Answer 5: `4040`

**User Interface**

Spark’s User Interface which shows the application's dashboard runs on which local port?

- 80
- 443
- 4040
- 8080

In [21]:
zones=pd.read_csv(zones_infile, nrows=10)

In [22]:
zones.dtypes

LocationID       int64
Borough         object
Zone            object
service_zone    object
dtype: object

In [23]:
zones_schema = types.StructType([
    types.StructField("LocationID", types.IntegerType(), True),
    types.StructField("Borough", types.StringType(), True),
    types.StructField("Zone", types.StringType(), True),
    types.StructField("service_zone", types.StringType(), True),
])

In [24]:
df_zones = spark.read \
    .option("header", "true") \
    .schema(zones_schema)\
    .csv(zones_infile)

In [25]:
df_zones.printSchema()

root
 |-- LocationID: integer (nullable = true)
 |-- Borough: string (nullable = true)
 |-- Zone: string (nullable = true)
 |-- service_zone: string (nullable = true)



In [26]:
df_zones.show(2)

+----------+-------+--------------+------------+
|LocationID|Borough|          Zone|service_zone|
+----------+-------+--------------+------------+
|         1|    EWR|Newark Airport|         EWR|
|         2| Queens|   Jamaica Bay|   Boro Zone|
+----------+-------+--------------+------------+
only showing top 2 rows



In [27]:
df_joined = df\
    .join(df_zones, df.PULocationID == df_zones.LocationID)\
    .where(F.col('Borough') != 'Unknown')

In [28]:
df_joined.printSchema()

root
 |-- dispatching_base_num: string (nullable = true)
 |-- pickup_datetime: timestamp (nullable = true)
 |-- dropOff_datetime: timestamp (nullable = true)
 |-- PULocationID: integer (nullable = true)
 |-- DOLocationID: integer (nullable = true)
 |-- SR_Flag: double (nullable = true)
 |-- Affiliated_base_number: string (nullable = true)
 |-- LocationID: integer (nullable = true)
 |-- Borough: string (nullable = true)
 |-- Zone: string (nullable = true)
 |-- service_zone: string (nullable = true)



In [29]:
df_joined\
    .groupBy('Zone')\
    .count()\
    .orderBy('count', ascending=True)\
    .first()[0]
    # .show(5)

'Jamaica Bay'

## Question 6: 

### Answer 6: `Jamaica Bay`

**Least frequent pickup location zone**

Load the zone lookup data into a temp view in Spark</br>
[Zone Data](https://github.com/DataTalksClub/nyc-tlc-data/releases/download/misc/taxi_zone_lookup.csv)

Using the zone lookup data and the FHV October 2019 data, what is the name of the LEAST frequent pickup location Zone?</br>

- East Chelsea
- Jamaica Bay
- Union Sq
- Crown Heights North



In [30]:
df.createOrReplaceTempView('view_fhv')
df_zones.createOrReplaceTempView('view_zones')

In [31]:
view_joined = spark.sql("""
SELECT 
    pickup_zone.Zone AS pickup_zone,
    COUNT(1) AS number_records
FROM
    view_fhv as fhv
INNER JOIN view_zones as pickup_zone
ON fhv.PULocationID = pickup_zone.LocationID
WHERE
    pickup_zone.Borough != 'Unknown'
GROUP BY
    pickup_zone
ORDER BY number_records
""")

In [32]:
view_joined.show(5)

+--------------------+--------------+
|         pickup_zone|number_records|
+--------------------+--------------+
|         Jamaica Bay|             1|
|Governor's Island...|             2|
| Green-Wood Cemetery|             5|
|       Broad Channel|             8|
|     Highbridge Park|            14|
+--------------------+--------------+
only showing top 5 rows

